### Conventional Transfer Learning and Simulation Testing ###

This notebook applies conventional transfer learning and simulation testing for the xBD holdout dataset.

In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import h5py

import shutil
from keras.models import load_model
from keras import layers
from keras import models
from keras import optimizers

In [2]:
model1 = load_model('cc_model_with_dropout_4.h5')
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 sequential (Sequential)     (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                  

In [3]:
# test_ds = tf.data.Dataset.load('turkey/test')
test_ds = tf.data.Dataset.load('xBD_Dataset/xBD_test_2')

history = model1.evaluate(test_ds)

74/74 [==============================] - 4s 8ms/step - loss: 0.1553 - precision: 0.7111 - recall: 0.5854 - accuracy: 0.9545


In [5]:
print(history)

[4.472379207611084, 0.14612868428230286, 0.3529411852359772, 0.7033833265304565]


In [13]:
test_predictions = model1.predict(test_ds, batch_size = 32)

326/326 [==============================] - 2s 6ms/step


In [42]:
model1 = load_model('cc_model_with_dropout_4.h5')
# Disassemble layers
layers = [l for l in model1.layers]


# Stack everything back

x = layers[0].output
for i in range(2, len(layers)):
    if i < len(layers)-12:
        layers[i].trainable = False
    else:
        layers[i].trainable = True
    x = layers[i](x)

# Final touch
new_model = tf.keras.Model(inputs=layers[0].input, outputs=x)
new_model.summary()
    
new_model.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.001*0.1),
                 metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.F1Score(),'accuracy'])



Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_input (InputLayer  [(None, 100, 100, 3)]    0         
 )                                                               
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0   

AttributeError: module 'keras.api._v2.keras.metrics' has no attribute 'F1Score'

In [38]:
# train_ds = tf.data.Dataset.load('turkey/train')
# val_ds = tf.data.Dataset.load('turkey/val')
train_ds = tf.data.Dataset.load('xBD_Dataset/xBD_train_2')
val_ds = tf.data.Dataset.load('xBD_Dataset/xBD_val_2')
test_ds = tf.data.Dataset.load('xBD_Dataset/xBD_test_2')

In [39]:
# len(list(test_ds[0]))

In [40]:
history1 = new_model.fit(
            train_ds,
            epochs=50,
            validation_data=val_ds,
#             class_weight={0:1,1:3}
            )

Epoch 1/50
6/6 [==============================] - 1s 59ms/step - loss: 0.4164 - precision_7: 1.0000 - recall_7: 0.7000 - accuracy: 0.8537 - val_loss: 0.4423 - val_precision_7: 0.9322 - val_recall_7: 0.6548 - val_accuracy: 0.7988
Epoch 2/50
6/6 [==============================] - 0s 19ms/step - loss: 0.3987 - precision_7: 0.9821 - recall_7: 0.6875 - accuracy: 0.8415 - val_loss: 0.4192 - val_precision_7: 0.9344 - val_recall_7: 0.6786 - val_accuracy: 0.8110
Epoch 3/50
6/6 [==============================] - 0s 20ms/step - loss: 0.3870 - precision_7: 0.9500 - recall_7: 0.7125 - accuracy: 0.8415 - val_loss: 0.4058 - val_precision_7: 0.9375 - val_recall_7: 0.7143 - val_accuracy: 0.8293
Epoch 4/50
6/6 [==============================] - 0s 22ms/step - loss: 0.3592 - precision_7: 1.0000 - recall_7: 0.7375 - accuracy: 0.8720 - val_loss: 0.3940 - val_precision_7: 0.9254 - val_recall_7: 0.7381 - val_accuracy: 0.8354
Epoch 5/50
6/6 [==============================] - 0s 20ms/step - loss: 0.3528 - prec

Epoch 37/50
6/6 [==============================] - 0s 19ms/step - loss: 0.2393 - precision_7: 1.0000 - recall_7: 0.8375 - accuracy: 0.9207 - val_loss: 0.3276 - val_precision_7: 0.9324 - val_recall_7: 0.8214 - val_accuracy: 0.8780
Epoch 38/50
6/6 [==============================] - 0s 20ms/step - loss: 0.2623 - precision_7: 0.9054 - recall_7: 0.8375 - accuracy: 0.8780 - val_loss: 0.3274 - val_precision_7: 0.9324 - val_recall_7: 0.8214 - val_accuracy: 0.8780
Epoch 39/50
6/6 [==============================] - 0s 20ms/step - loss: 0.2584 - precision_7: 0.9855 - recall_7: 0.8500 - accuracy: 0.9207 - val_loss: 0.3271 - val_precision_7: 0.9324 - val_recall_7: 0.8214 - val_accuracy: 0.8780
Epoch 40/50
6/6 [==============================] - 0s 20ms/step - loss: 0.2641 - precision_7: 0.9714 - recall_7: 0.8500 - accuracy: 0.9146 - val_loss: 0.3264 - val_precision_7: 0.9324 - val_recall_7: 0.8214 - val_accuracy: 0.8780
Epoch 41/50
6/6 [==============================] - 0s 19ms/step - loss: 0.2670 -

In [41]:
history = new_model.evaluate(test_ds)

74/74 [==============================] - 0s 6ms/step - loss: 0.2315 - precision_7: 0.5641 - recall_7: 0.8049 - accuracy: 0.9431


In [11]:
counter = 0
for element in test_ds:
    preds = model1.predict(element[0])
    for i in range(element[0].shape[0]):
        img = PIL.Image.fromarray(element[0][i,:,:,:].numpy().astype(np.uint8))
        if preds[i] > 0.5:
            dstatus = 'turkey/test_predictions/undamaged/'
        else:
            dstatus = 'turkey/test_predictions/damaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')

1/1 [==============================] - 0s 65ms/step


In [ ]:
counter

In [56]:
counter = 0
for element in test_ds:
    preds = new_model.predict(element[0])
    for i in range(element[0].shape[0]):
        img = PIL.Image.fromarray(element[0][i,:,:,:].numpy().astype(np.uint8))
        if preds[i] > 0.5:
            dstatus = 'xBD_Dataset/test_predictions/damaged/'
        else:
            dstatus = 'xBD_Dataset/test_predictions/undamaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')

1/1 [==============================] - 0s 19ms/step


In [13]:
counter

2353

In [69]:
counter = 0
dataset = test_ds.enumerate()
arr = []
for element in dataset.as_numpy_iterator():
    for i in range(len(element[1][1])):
        dmg = element[1][1][i]   
        img = PIL.Image.fromarray(element[1][0][i].astype(np.uint8))
        if dmg == 1:
            dstatus = 'xBD_Dataset/test_truth/damaged/'
        else:
            dstatus = 'xBD_Dataset/test_truth/undamaged/'
        counter += 1
        img.save(dstatus + str(counter) + '.png')